In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import json
from pathlib import Path
from typing import List

import cv2
import dataclass_array as dca
import matplotlib.pyplot as plt
import mitsuba as mi
import numpy as np
from PIL import Image
import pycolmap
import pyrender
import transforms3d as t3d
import trimesh
import visu3d as v3d

import burybarrel.colmap_util as cutil
from burybarrel.image import render_v3d, imgs_from_dir

In [ ]:
def temporal_window(imgs, idx, window_size):
    if window_size % 2 == 0:
        window_size += 1
    half = window_size // 2
    if idx >= half and idx < len(imgs) - half:
        return imgs[idx - half:idx + half + 1]
    elif idx >= len(imgs) - half:
        return imgs[-window_size:]
    else:
        return imgs[:window_size]

In [ ]:
temporal_window([0,1,2,3,4,5,6,7,8,9], 9, 5)

In [ ]:
outdir = Path("../results/dive3-depthcharge-03-04-trimmed-reconstr/corrected")
imgdir = Path("../data/dive-data/Dive3/clips/dive3-depthcharge-03-04-trimmed")
imgpaths, imgs = imgs_from_dir(imgdir, asarray=True)

$$R(x,y)=\overline{\bar{f}(x,y)}\cdot\min(\frac{1}{\bar{f}(x,y)},\frac{\text{Maxscale}}{\bar{f}(x,y)+4\sigma(x,y)})$$

In [ ]:
for i, img in enumerate(imgs):
    windowimg = temporal_window(imgs, i, 20)
    avgpx = np.mean(windowimg, axis=0)
    stdpx = np.std(windowimg, axis=0)
    avgall = np.mean(windowimg)
    maxscale = 255
    correction = avgall * np.min([1 / avgpx, maxscale / (avgpx + 4 * stdpx)], axis=0)
    correctedimg = Image.fromarray((imgs[i] * correction).astype(np.uint8))
    correctedimg.save(outdir / imgpaths[i].name)

In [ ]:
Image.fromarray((imgs[0] * correction).astype(np.uint8))